# Generative networks

Recurrent Neural Networks (RNNs) and their gated cell variants such as Long Short-Term Memory cells (LSTMs) and Gated Recurrent Units (GRUs) provide a mechanism for language modeling &mdash; they can learn word ordering and make predictions for the next word in a sequence. This allows us to use RNNs for **generative tasks**, such as ordinary text generation, machine translation, and even image captioning.

In the RNN architecture we discussed in the previous notebook, each RNN unit produced the next hidden state as an output. However, we could also add another output to each recurrent unit, which would allow us to output a **sequence** with the same length as the original sequence. Moreover, we could use RNN units that don't accept an input at each step, instead they take an initial state vector and produce a sequence of outputs.

This allows for different neural architectures, as is shown in the picture below:

循环神经网络 (RNN) 及其门控单元变体，例如长短期记忆单元 (LSTM) 和门控循环单元 (GRU)，为语言建模提供了一种机制—— 他们可以学习词序并预测序列中的下一个词。 这使我们能够将 RNN 用于**生成任务**，例如普通文本生成、机器翻译，甚至图像字幕。

在我们在之前的笔记本中讨论的 RNN 架构中，每个 RNN 单元产生下一个隐藏状态作为输出。 然而，我们也可以向每个循环单元添加另一个输出，这将允许我们输出一个与原始序列长度相同的**序列**。 此外，我们可以使用不在每一步都接受输入的 RNN 单元，而是采用初始状态向量并生成一系列输出。

这允许不同的神经架构，如下图所示：

![Image showing common recurrent neural network patterns.](./images/unreasonable-effectiveness-of-rnn.jpg)
*Image from blog post [Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by [Andrej Karpaty](http://karpathy.github.io/)*

* **One-to-one** is a traditional neural network with one input and one output.
* **One-to-many** is a generative architecture that accepts one input value, and generates a sequence of output values. For example, if we want to train an **image captioning** network that produces a textual description of a picture, we can use a picture as input, pass it through a CNN to obtain the hidden state, and then have a recurrent chain generate the caption word-by-word.
* **Many-to-one** corresponds to the RNN architectures we described in the previous unit, such as text classification.
* **Many-to-many**, or **sequence-to-sequence** corresponds to tasks such as **machine translation**, in which case the RNN first collects all the information from the input sequence into the hidden state, and then unrolls this state into the output sequence.

In this unit, we will focus on simple generative models that help us generate text. For simplicity, let's build a **character-level network**, which generates text letter by letter. During training, we need to take a text corpus, and split it into letter sequences. 

> For the sandbox environment, we need to run the following cell to make sure the required library is installed, and data is prefetched. If you're running locally, you can skip the following cell.


* **一对一**是一种传统的神经网络，具有一个输入和一个输出。
* **一对多** 是一种生成架构，它接受一个输入值，并生成一系列输出值。 例如，如果我们想训练一个生成图片文本描述的**图像字幕**网络，我们可以使用图片作为输入，将其传递给 CNN 以获得隐藏状态，然后有一个循环链 逐字生成标题。
* **多对一** 对应我们在上一单元中描述的 RNN 架构，例如文本分类。
* **多对多**，或者**sequence-to-sequence**对应于**机器翻译**等任务，在这种情况下，RNN首先将输入序列中的所有信息收集到隐藏层中 状态，然后将此状态展开到输出序列中。

在本单元中，我们将专注于帮助我们生成文本的简单生成模型。 为简单起见，让我们构建一个**字符级网络**，逐个字母地生成文本。 在训练期间，我们需要获取文本语料库，并将其拆分为字母序列。

> 对于沙盒环境，我们需要运行以下单元以确保安装了所需的库，并预取了数据。 如果您在本地运行，则可以跳过以下单元格。

In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Building character vocabulary

To build a character-level generative network, we need to split the text into individual characters instead of words. The `TextVectorization` layer that we've been using can't do that, so we have two options:

* Manually load text and implement our own tokenization, as in [this official Keras example](https://keras.io/examples/generative/lstm_character_level_text_generation/).
* Use the `Tokenizer` class for character-level tokenization.

We'll go with the second option. `Tokenizer` can also be used to tokenize into words, so we should be able to switch from character-level to word-level tokenization quite easily.

To do character-level tokenization, we need to pass the `char_level=True` parameter:


要构建字符级生成网络，我们需要将文本拆分为单个字符而不是单词。 我们一直在使用的 `TextVectorization` 层无法做到这一点，因此我们有两个选择：

* 手动加载文本并实现我们自己的标记化，如 [此官方 Keras 示例](https://keras.io/examples/generative/lstm_character_level_text_generation/)。
* 使用 `Tokenizer` 类进行字符级分词。

我们将选择第二个选项。 `Tokenizer` 也可以用来标记词，所以我们应该能够很容易地从字符级切换到单词级标记化。

要进行字符级标记化，我们需要传递 `char_level=True` 参数：

In [3]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

We also want to use one special token to denote **end-of-sequence**, which we'll call `<eos>`. Let's add it manually to the vocabulary:

我们还想使用一个特殊标记来表示 **序列结束**，我们将其称为 `<eos>`。 让我们手动将其添加到词汇表中：

In [4]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Now, to encode text into sequences of numbers, we can use:

现在，要将文本编码为数字序列，我们可以使用：

In [5]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Training a generative RNN to generate titles

Here's how we'll train an RNN to generate titles: on each step, we'll take one title as input, and for each input character in that title, we'll train the network to generate the next character as output:

以下是我们如何训练 RNN 生成标题：在每一步中，我们将一个标题作为输入，对于该标题中的每个输入字符，我们将训练网络生成下一个字符作为输出：

![Image showing an example RNN generation of the word 'HELLO'.](./images/rnn-generate.png)

For the last character of our sequence, we'll ask the network to generate the `<eos>` token.

The main difference between the generative RNN we're using here and the RNNs we've seen previously is that each cell of the generative RNN will produce an output, not just the final cell. This can be achieved by specifying the `return_sequences` parameter of the RNN cell.

Thus, during training, an input to the network would be a sequence of encoded characters of some length, and the corresponding output would be a sequence of the same length, but shifted by one element and terminated by `<eos>`. The minibatch will consist of several such sequences, and we'll need to use **padding** to align all sequences.

Let's create functions that transform the dataset for us. Because we want to pad sequences at the minibatch level, we'll first create the minibatches by calling `.batch()`, and then use `map` to do transformation. Here's the code for the function that does the transformation, which takes a whole minibatch as a parameter:

对于序列的最后一个字符，我们将要求网络生成 `<eos>` 标记。

我们在这里使用的生成 RNN 与我们之前看到的 RNN 之间的主要区别在于，生成 RNN 的每个单元都会产生一个输出，而不仅仅是最终单元。 这可以通过指定 RNN 单元的`return_sequences`参数来实现。

因此，在训练期间，网络的输入将是一定长度的编码字符序列，相应的输出将是相同长度的序列，但移动一个元素并以“<eos>”终止。 minibatch 将包含几个这样的序列，我们需要使用 **padding** 来对齐所有序列。

让我们创建为我们转换数据集的函数。 因为我们想在小批量级别填充序列，所以我们首先通过调用 `.batch()` 来创建小批量，然后使用 `map` 进行转换。 下面是执行转换的函数的代码，它将整个小批量作为参数：

In [6]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

A few important things that we do here:
* We first extract the actual text from the string tensor.
* `text_to_sequences` converts the list of strings into a list of integer tensors.
* `pad_sequences` pads those tensors to their maximum length.
* We finally one-hot encode all the characters, and also do the shifting and `<eos>` appending. We will soon see why we need one-hot-encoded characters.

However, this function is **Pythonic** &mdash; it cannot be automatically translated into the TensorFlow computational graph. (Computational graphs will be discussed in more detail in the module on going beyond Keras of this learning path.) If we try to pass it directly to the `Dataset.map` function, which is how we intend to use this function, we'll get an error. To get around this problem, we need to enclose this Pythonic call by using the `py_function` wrapper: 

我们在这里做的一些重要的事情：
* 我们首先从字符串张量中提取实际文本。
* `text_to_sequences` 将字符串列表转换为整数张量列表。
* `pad_sequences` 将这些张量填充到它们的最大长度。
* 我们最终对所有字符进行了一次热编码，并进行了移位和 `<eos>` 追加。 我们很快就会明白为什么我们需要 one-hot-encoded 字符。

然而，这个函数是 **Pythonic** &mdash; 它不能自动转换为 TensorFlow 计算图。 （计算图将在本学习路径的超越 Keras 的模块中更详细地讨论。）如果我们尝试将它直接传递给`Dataset.map`函数，这就是我们打算如何使用这个函数，我们“ 会得到一个错误。 为了解决这个问题，我们需要使用 `py_function` 包装器来封装这个 Pythonic 调用：

In [7]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: You may be wondering if instead of calling the `py_function` wrapper we could transform the dataset using standard Python functions before passing it to `fit`. While this can definitely be done, when using `Dataset.map` the data transformation pipeline is executed using TensorFlow's computational graph, which takes advantage of GPU computations and minimizes the need to pass data between CPU/GPU.

We can now build our generator network and start training. Our network can be based on any recurrent cell which we discussed in the previous unit (simple, LSTM or GRU). In our example we'll use LSTM.

Because the network takes characters as input, the vocabulary size is pretty small. Therefore we don't need an embedding layer &mdash; we can feed one-hot-encoded input directly into the LSTM cell. The output layer is then a `Dense` classifier that will convert the LSTM output into one-hot-encoded token numbers.

In addition, since we're dealing with variable-length sequences, we can use a `Masking` layer to create a mask that will ignore the padded part of the string. This is not strictly needed, because we're not interested in everything that goes beyond the `<eos>` token, but we will use it to get some experience with this layer type. The input shape is `(None, vocab_size)`, where `None` indicates the sequence of variable length, and the output shape is also `(None, vocab_size)`, as you can see from the `summary` printout below: 

> **注意**：您可能想知道我们是否可以在将数据集传递给 `fit` 之前使用标准 Python 函数转换数据集，而不是调用 `py_function` 包装器。 虽然这绝对可以做到，但在使用`Dataset.map`时，数据转换管道是使用 TensorFlow 的计算图执行的，它利用了 GPU 计算并最大限度地减少了在 CPU/GPU 之间传递数据的需要。

我们现在可以构建生成器网络并开始训练。 我们的网络可以基于我们在上一单元中讨论过的任何循环单元（简单的、LSTM 或 GRU）。 在我们的示例中，我们将使用 LSTM。

因为网络以字符作为输入，所以词汇量非常小。 因此我们不需要嵌入层—— 我们可以将 one-hot-encoded 输入直接输入到 LSTM 单元中。 然后输出层是一个`Dense`分类器，它将 LSTM 输出转换为单热编码的标记数。

此外，由于我们处理的是可变长度序列，我们可以使用`Masking`层来创建一个掩码，该掩码将忽略字符串的填充部分。 这不是严格需要的，因为我们对超出 `<eos>` 标记的所有内容都不感兴趣，但我们将使用它来获得有关此层类型的一些经验。 输入形状为`(None, vocab_size)`，其中`None`表示变长序列，输出形状也是`(None, vocab_size)`，从下面的`summary`打印输出可以看出：

In [8]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 273s 18ms/step - loss: 1.5460


## Generating output

Now that we've trained the model, we want to use it to generate output. First of all, we need a way to decode text represented by a sequence of token numbers. We could use the `tokenizer.sequences_to_texts` function, but it doesn't work well with character-level tokenization. Therefore we will take a dictionary of tokens from the tokenizer (called `word_index`), build a reverse map, and write our own decoding function:

现在我们已经训练了模型，我们想用它来生成输出。 首先，我们需要一种方法来解码由一系列标记数字表示的文本。 我们可以使用 `tokenizer.sequences_to_texts` 函数，但它不适用于字符级标记化。 因此，我们将从分词器（称为 `word_index` ）中获取一个分词字典，构建一个反向映射，并编写我们自己的解码函数：

In [9]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map.get(t,'') for t in x])

Now, let's do the generation. We first encode a string passed as parameter into a sequence, and then on each step we call our network to infer the next character. 

The output of the network is a vector of `vocab_size` elements representing probablities of each token, and we can find the most probably token number by using `argmax`. We then append this character to the generated list of tokens, and proceed with the generation. This process of generating one character is repeated `size` times to generate the required number of characters, and we terminate early if `eos_token` is encountered.

现在，让我们做这一代。 我们首先将作为参数传递的字符串编码为一个序列，然后在每一步中调用我们的网络来推断下一个字符。

网络的输出是表示每个标记概率的 `vocab_size` 元素向量，我们可以使用 `argmax` 找到最有可能的标记数。 然后我们将这个字符附加到生成的标记列表中，并继续生成。 这个生成一个字符的过程会重复 `size` 次以生成所需数量的字符，如果遇到 `eos_token`，我们会提前终止。

In [10]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s Server Strike Street Start (AP)'

## Sampling output during training

Because we don't have any useful metrics such as *accuracy*, the only way we can see that our model is getting better is by **sampling** generated strings during training. We use **callbacks** for that, which are functions that we can pass to the `fit` function, and that will be called periodically during training. 

因为我们没有任何有用的指标，例如 *accuracy*，我们可以看到我们的模型变得更好的唯一方法是在训练期间对生成的字符串进行**采样**。 我们为此使用**回调**，这是我们可以传递给 `fit` 函数的函数，它将在训练期间定期调用。

In [11]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 274s 18ms/step - loss: 1.2743
Epoch 2/3
15000/15000 [==============================] - 274s 18ms/step - loss: 1.2075
Epoch 3/3
15000/15000 [==============================] - 274s 18ms/step - loss: 1.1763


This example already generates some pretty good text, but it can be further improved in several ways:
* **More text**. We only used titles for our task, but you may want to experiment with full text. Remember that RNNs aren't that great at handling long sequences, so it makes sense to either split them into shorter sentences, or to always train on a sequence of fixed length `num_chars` (say, 256). You may try to change the example above into this architecture, using the [official Keras tutorial](https://keras.io/examples/generative/lstm_character_level_text_generation/) as inspiration.
* **Multilayer LSTM**. It makes sense to try 2 or 3 layers of LSTM cells. As we mentioned in the previous unit, each layer of LSTM extracts certain patterns from the text, and in the case of a character-level generator we can expect lower LSTM levels to be responsible for extracting syllables, and higher levels to extract words and word combinations. This can be implemented by adding layers sequentially to the model.
* You may also want to experiment with **GRU units** and see which one performs better, and with **different hidden layer sizes**. A hidden layer that's too large may result in overfitting (because the network will learn the exact text), and a hidden layer that's too small might not produce good results.

这个例子已经生成了一些非常好的文本，但它可以通过多种方式进一步改进：
* **更多文字**。 我们只为我们的任务使用了标题，但您可能想尝试使用全文。 请记住，RNN 不太擅长处理长序列，因此将它们拆分成较短的句子或始终在固定长度“num_chars”（例如 256）的序列上进行训练是有意义的。 您可以尝试将上面的示例更改为此架构，使用 [官方 Keras 教程](https://keras.io/examples/generative/lstm_character_level_text_generation/) 作为灵感。
* **多层 LSTM**。 尝试 2 层或 3 层 LSTM 单元是有意义的。 正如我们在上一单元中提到的，LSTM 的每一层都从文本中提取某些模式，在字符级生成器的情况下，我们可以预期较低的 LSTM 层负责提取音节，而较高的层负责提取单词和单词 组合。 这可以通过向模型顺序添加层来实现。
* 您可能还想尝试使用 **GRU 单元**，看看哪个表现更好，以及**不同的隐藏层大小**。 太大的隐藏层可能会导致过度拟合（因为网络会学习到准确的文本），而太小的隐藏层可能不会产生好的结果。

## Soft text generation and temperature

In the code for the `generate` function, we took the character with the highest probability as the next character in the generated text. This resulted in text that cycles between the same character sequences again and again, like in this example:

在 `generate` 函数的代码中，我们将概率最高的字符作为生成文本中的下一个字符。 这导致文本在相同的字符序列之间一次又一次地循环，就像这个例子：

```
today of the second the company and a second the company ...
```

However, if we look at the probability distribution for the next character, it may be that there are several high probabilities that are pretty similar. For example, when looking for the next character in the sequence '*play*', it's similarly likely that it's either space or **e** (as in the word *player*).

Therefore, it's not always the best choice to select the character with the absolute highest probability &mdash; choosing the second or third highest might still lead to meaningful text, and may avoid cycling through character sequences. Therefore, a better strategy is to **sample** characters from the probability distribution given by the network output.

This sampling can be done using the `np.multinomial` function which implements a **multinomial distribution**. A function that implements this **soft** text generation is defined below:

但是，如果我们查看下一个字符的概率分布，可能有几个非常相似的高概率。 例如，在查找序列“*play*”中的下一个字符时，它同样可能是空格或 **e**（如单词 *player* 中）。

因此，以绝对最高概率选择字符并不总是最佳选择&mdash; 选择第二或第三高的可能仍然会导致有意义的文本，并且可能避免循环字符序列。 因此，更好的策略是从网络输出给出的概率分布中**采样**字符。

可以使用实现**多项分布**的`np.multinomial`函数来完成此采样。 下面定义了一个实现这个**soft**文本生成的函数：

In [12]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strong to be in Iraq line at US first profit
On Sunday DS #39; Market Shot to Expect (AP)
Moscow, SP completes street to share maker straight talks
President Sutel to Return in The Takeover Three Street (AP)
Little red riding hood for talks to hit on to start in Iraq

--- Temperature = 0.8
Today olies safel fate withdraws to leave  #39;Profit #39; smathe off the meet
On Sunday BC - Miscripys Southern dead in Iraq
Moscow, SP to up air burine with Mart oppositive, MySWilliers price stand
President Israli Wasted Predemation Retailers for Mondain Convent
Little red riding hood field skyallingnaul by buys

--- Temperature = 1.0
Today casop Symantec family start worries With Montreal evi
On Sunday DA in Loest Piofut For Afghan Minister (AP)
Moscow, S TSVRCIA Forting Have Black Chapfers #39; In Ractor
President iveshight arophis, remain from collemen shipal back (AFP)
Little red riding hood gin calp sold, not to target

--- Temperature = 1.3
Today S economy 

We've introduced one more parameter called **temperature**, which indicates how strongly we should choose higher probability characters over lower probability ones. If the temperature is close to 0, we choose the highest probability character, and when the temperature approaches infinity then all probabilities become equal, and we randomly select the next character. In the example above we can observe that the output becomes meaningless when we increase the temperature too much, and starts cycling when we lower it closer to 0. 

我们还引入了一个名为**temperature**的参数，它表明我们应该在多大程度上选择高概率的字符而不是低概率的字符。如果temperature接近0，我们就选择概率最高的字符，而当temperature接近无穷大时，所有的概率就变得相等，我们就随机选择下一个字符。在上面的例子中，我们可以观察到，当我们把temperature提高得太多时，输出就变得毫无意义，而当我们把temperature降低到接近0时，就开始循环了。